# 運用系処理

## ライブラリインポート

In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from elastic_manager.elastic_manager import ElasticManager
import pandas as pd
pd.set_option('display.max_rows', 1000) # 最大表示数

## インデックス一覧
インデックスの名前とドキュメント数、サイズ（kB）を一覧で表示します。  
indexを指定しない場合、すべてのインデックスが表示されます。  
indexを指定した場合、そのパターンに一致するインデックス一覧が表示されます。  
例）ElasticManager.show_indices(index="rawdata-*")  
⇒ rawdataインデックスの一覧が表示されます。

In [7]:
# 例）ElasticManager.show_indices(index="rawdata-*")
ElasticManager.show_indices(index="shots-*-data")

,index,docs.count,store.size
0,shots-20201201010000-data,9281299,1199088
1,shots-20210129114815-data,0,0


## インデックス削除
DELETE_INDEXに削除するインデックス名を入力します。ワイルドカード(\*)が使用可能です。  
例）rawdata-20201201\*  
⇒ 2020/12/01のrawdataデータインデックスがすべて削除されます。

In [8]:
DELETE_INDEX = 'shots-20210129114815-data'

In [9]:
ElasticManager.delete_index(DELETE_INDEX)

## データ削除（連番指定）
連番指定でデータを削除します。以下のパラメータを指定してください。

* TARGET_INDEX：データ削除対象のインデックス名  
* START_SEQ_NUM：開始番号  
* END_SEQ_NUM：終了番号

In [8]:
TARGET_INDEX = 'rawdata-20210118153002'
START_SEQ_NUM = 0
END_SEQ_NUM = 10

In [9]:
ElasticManager.delete_data_by_seq_num(index=TARGET_INDEX, start=START_SEQ_NUM, end=END_SEQ_NUM)

## データ削除（ショット番号指定）
ショット番号指定でデータを削除します。以下のパラメータを指定してください。  

* TARGET_INDEX：データ削除対象のインデックス名  
* SHOT_NUMBER：ショット番号

In [5]:
TARGET_INDEX = 'shots-20201201010000-data'
SHOT_NUMBER = 2

In [6]:
ElasticManager.delete_data_by_shot_num(index=TARGET_INDEX, shot_number=SHOT_NUMBER)

# テンポラリファイル読み込み

In [8]:
import glob

# テンポラリファイル読み込み
target_dir = "/home/ymiyamoto5/h-one-experimental-system/shared/data/20210201151814"

pickle_file_list = glob.glob(os.path.join(target_dir, "*.pkl"))
pickle_file_list.sort()

rawdata_df = pd.DataFrame()

for pickle_file in pickle_file_list:
    pickle_df = pd.read_pickle(pickle_file)
    rawdata_df = pd.concat([rawdata_df, pickle_df], axis=0)

rawdata_df.reset_index()
print(f"データ数: {len(rawdata_df)}")

データ数: 6154240


In [9]:
rawdata_df.head()

,sequential_number,timestamp,displacement,load01,load02,load03,load04
0,0,1.608073e+09,5.051,-1.598,-1.598,-1.598,-1.598
1,1,1.608073e+09,5.051,-1.709,-1.709,-1.710,-1.710
2,2,1.608073e+09,5.051,-1.813,-1.813,-1.813,-1.813
3,3,1.608073e+09,5.052,-1.911,-1.911,-1.912,-1.912
4,4,1.608073e+09,5.052,-2.001,-2.001,-2.001,-2.001


# テンポラリファイル データ削除

In [10]:
rawdata_df_fixed = rawdata_df[10:] # 最初の10サンプル以外を取り出し
rawdata_df_fixed

,sequential_number,timestamp,displacement,load01,load02,load03,load04
10,10,1.608073e+09,5.051,-2.354,-2.355,-2.355,-2.355
11,11,1.608073e+09,5.051,-2.379,-2.379,-2.380,-2.380
12,12,1.608073e+09,5.052,-2.398,-2.398,-2.398,-2.398
13,13,1.608073e+09,5.050,-2.407,-2.407,-2.407,-2.407
14,14,1.608073e+09,5.051,-2.408,-2.408,-2.408,-2.408
...,...,...,...,...,...,...,...
162811,6154235,1.608073e+09,4.862,-2.391,-2.391,-2.391,-2.391
162812,6154236,1.608073e+09,4.709,-2.366,-2.366,-2.366,-2.366
162813,6154237,1.608073e+09,4.559,-2.335,-2.335,-2.335,-2.335
162814,6154238,1.608073e+09,4.407,-2.292,-2.292,-2.293,-2.293


## テンポラリファイル出力

In [11]:
rawdata_df_fixed.to_pickle("/home/ymiyamoto5/h-one-experimental-system/shared/data/20210201151814/fixed.pkl")